## Импорт библиотек

In [ ]:
import os
import gc
import pickle
import time

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor

from joblib import dump
from joblib import load

import lightgbm as lgb
import catboost as cb

import optuna

import matplotlib.pyplot as plt

In [ ]:
# Время старта работы ноутбука
notebook_starttime = time.time()

## Настройка: сабмит или локально

In [ ]:
# Ставим is_local в True, если локально работаем, если сабмитим - ставим в False
#is_local = False
is_local = True

# Ставим is_gpu в True, если будем работать на GPU, если на процессоре - ставим в False
is_gpu = False
#is_gpu = True

# Ставим only_one_model = True, если обучается только дна модель без ансбамбля.
# only_one_model = False
only_one_model = True

# Раз во сколько циклов сабмита (дней) учить модели заново с учетом новый данных, добавленных к старым
# Скажем если учить заново каждые 30 циклов (после каждых 30 предсказанных дней) то ставим 30
learn_again_period = 100

In [ ]:
class MonthlyKFold:
    def __init__(self, n_splits=3):
        self.n_splits = n_splits
        
    def split(self, X, y, groups=None):
        dates = 12 * X["year"] + X["month"]
        timesteps = sorted(dates.unique().tolist())
        X = X.reset_index()
        
        for t in timesteps[-self.n_splits:]:
            idx_train = X[dates.values < t].index
            idx_test = X[dates.values == t].index
            
            yield idx_train, idx_test
            
    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

## Feature Engineering sub

In [ ]:
def feature_eng(df_data, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target):
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_client = (
        df_client
        .with_columns(
            (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
        )
    )
    
    df_gas = (
        df_gas
        .rename({"forecast_date": "date"})
        .with_columns(
            (pl.col("date") + pl.duration(days=1)).cast(pl.Date)
        )
    )
    
    df_electricity = (
        df_electricity
        .rename({"forecast_date": "datetime"})
        .with_columns(
            pl.col("datetime") + pl.duration(days=1)
        )
    )
    
    df_location = (
        df_location
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    df_forecast = (
        df_forecast
        .rename({"forecast_datetime": "datetime"})
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            #pl.col('datetime').dt.convert_time_zone("Europe/Bucharest").dt.replace_time_zone(None).cast(pl.Datetime("us")),
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
            #pl.col('datetime').cast(pl.Datetime)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical = (
        df_historical
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_forecast_date = (
        df_forecast
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_forecast_local = (
        df_forecast
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_historical_date = (
        df_historical
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_historical_local = (
        df_historical
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    # Объединение всех обработанных данных с основным датафреймом df_data
    df_data = (
        df_data
        .join(df_gas, on="date", how="left")
        .join(df_client, on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity, on="datetime", how="left")
        
        .join(df_forecast_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_historical_date, on="datetime", how="left", suffix="_hd")
        .join(df_historical_local, on=["county", "datetime"], how="left", suffix="_hl")
        
        .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fdw")
        .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_flw")
        .join(df_historical_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hdw")
        .join(df_historical_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hlw")
        
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        # Создание категориальных признаков и тригонометрических функций времени
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"), # Добавление номера дня в году
            pl.col("datetime").dt.hour().alias("hour"),# Добавление часа
            pl.col("datetime").dt.day().alias("day"),# Добавление дня
            pl.col("datetime").dt.weekday().alias("weekday"),# Добавление дня недели
            pl.col("datetime").dt.month().alias("month"),# Добавление месяца
            pl.col("datetime").dt.year().alias("year"),# Добавление года
        )
        # Приведение типов данных
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("category_1"),
        )
        
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"), # Тригонометрические функции для дня в году
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
         # Удаление ненужных колонок
        .drop("date", "datetime", "hour", "dayofyear")
    )
    
    # return df_data, df_historical_local
    return df_data

In [ ]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "category_1"]
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    
    return df

In [ ]:
# для оптуны
def lgb_objective(trial):
    params = {
        'n_iter'           : 1000,
        'verbose'          : -1,
        'random_state'     : 42,
        'objective'        : 'l2',
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.5, 1.0),
        'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 4, 256),
        'max_depth'        : trial.suggest_int('max_depth', 5, 10),
        'max_bin'          : trial.suggest_int('max_bin', 32, 1024),
    }
    
    model  = lgb.LGBMRegressor(**params)
    X, y   = df_train.drop(columns=["target"]), df_train["target"]
    cv     = MonthlyKFold(1)
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    return -1 * np.mean(scores)

## Global Variables

In [ ]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

# Для локальных вычислений. Последний data_block_id тренировочной выборки
# А начиная со следующего data_block_id и до конца идет тест
# train_end_data_block_id = 500
train_end_data_block_id = 600

data_cols        = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id', 'data_block_id']
# В df_data_cols колонки в таком порядке в каком они потом формируются в df_data
df_data_cols     = ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'datetime', 'data_block_id', 'row_id']
client_cols      = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_cols         = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_cols = ['forecast_date', 'euros_per_mwh']
forecast_cols    = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols  = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols    = ['longitude', 'latitude', 'county']
target_cols      = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

save_path = None
load_path = None

## Исследование

In [ ]:
# Загрузка данных об энергопотреблении
train = pd.read_csv(os.path.join(root, "train.csv"))

# Создание сводной таблицы с средними значениями целевой переменной (target)
# для каждой комбинации даты, округа, типа продукта, бизнеса и потребления
pivot_train = train.pivot_table(
    index='datetime',
    columns=['county', 'product_type', 'is_business', 'is_consumption'],
    values='target',
    aggfunc='mean'
)

# Переименование колонок для удобства доступа и интерпретации
pivot_train.columns = ['county{}_productType{}_isBusiness{}_isConsumption{}'.format(*col) for col in pivot_train.columns.values]
pivot_train.index = pd.to_datetime(pivot_train.index)

pivot_train

### 2023 год 

In [ ]:
# Копирование сводной таблицы для визуализации
df_plot = pivot_train.copy()

# Нормализация данных для визуализации
df_plot = (df_plot - df_plot.min()) / (df_plot.max() - df_plot.min())

# Ресемплирование данных по дням и вычисление средних значений
df_plot_resampled_D = df_plot.resample('D').mean()

# Визуализация нормализованных данных с прозрачностью (alpha=0.1)
df_plot_resampled_D.loc['2022-7':].plot(alpha=0.1, color='green', figsize=(18, 6), legend=False)


In [ ]:
# Выбор колонок, соответствующих различным категориям потребления
columns_consumption_0 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption0')]
columns_consumption_1 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption1')]

# Создание фигуры для визуализации
plt.figure(figsize=(15, 6))

# Создание пустых линий для легенды
plt.plot([], color='red', label='is_Consumption = 1')  # Изменено на желтый цвет
plt.plot([], color='black', label='is_Consumption = 0')   # Изменено на черный цвет

# Отображение легенды
plt.legend()

# Визуализация данных для 'is_Consumption = 0' черным цветом
for column in columns_consumption_0:
    df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='black', legend=False)  # Изменено на черный

# Визуализация данных для 'is_Consumption = 1' желтым цветом
for column in columns_consumption_1:
    df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='red', legend=False)  # Изменено на желтый

# Отображение графика
plt.show()




## Подготовка данных

### Запись тестовых и тренировочных csv файлов

In [ ]:
if is_local:
    # Если выполняем локально
    train_path = 'train'
    if not os.path.exists(train_path):
        # Создание каталога, если его нет
        os.makedirs(train_path)
    test_path = 'example_test_files'
    if not os.path.exists(test_path):
        # Создание каталога, если его нет
        os.makedirs(test_path)
else:
    # Если сабмит
    train_path = root
# Путь, куда запишем csv файлы для теста

In [ ]:
# Разделяет датафрейм на тренировочную и тестовую часть
# Возвращает часть датафрейма для тренировки, тестовую часть датафрейма записывает в каталог с тестами
def split_train_test(filename):
    df = pd.read_csv(os.path.join(root, filename))
    
    #Запишем часть данных для теста
    test_df = df[df["data_block_id"] > train_end_data_block_id]
    if (filename =="train.csv"):
        # Берем только те ячейки где target был не нулевым
        test_df = test_df[test_df["target"].notnull()]
        
    test_df.to_csv(os.path.join(test_path, filename), index=False)

    #Запишем часть данных для трейна
    train_df = df[df["data_block_id"] <= train_end_data_block_id]
    train_df.to_csv(os.path.join(train_path, filename), index=False)

# Доводим до ума тестовые таблицы чтобы они были точно такие как в реальном сабмите
def test_dfs_tune():
    # Делаем таблицу revealed_targets.csv
    df = pd.read_csv(os.path.join(test_path, "train.csv"))
    df.to_csv(os.path.join(test_path, 'revealed_targets.csv'), index=False)
    
    # Делаем таблицу test.csv
    df.rename(columns={'datetime': 'prediction_datetime'}, inplace=True)
    df.drop('target', axis=1, inplace=True)
    df['currently_scored'] = False
    df.to_csv(os.path.join(test_path, 'test.csv'), index=False)
    
    # Делаем таблицу sample_submission.csv
    selected_columns = ['row_id', 'data_block_id']
    df = df[selected_columns]
    df['target'] = 0
    df.to_csv(os.path.join(test_path, 'sample_submission.csv'), index=False)

# Сборка разделения файлов
def make_split():
    # csv файлы которые будем делить:
    csv_names = ["train.csv", "client.csv", "gas_prices.csv", "electricity_prices.csv", "forecast_weather.csv", "historical_weather.csv"]
    for csv_name in csv_names:
        split_train_test(csv_name)
    # Доделываем тестовые таблицы
    test_dfs_tune()

In [ ]:
%%time

# Создаем файлы csv c тренировочными и тестовыми таблицами
if is_local:
    # Пока отключил создание тестовых файлом. У меня локально они есть
    make_split()
    pass

### Data I/O

In [ ]:
%%time
df_data        = pl.read_csv(os.path.join(train_path, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client      = pl.read_csv(os.path.join(train_path, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas         = pl.read_csv(os.path.join(train_path, "gas_prices.csv"), columns=gas_cols, try_parse_dates=True)
df_electricity = pl.read_csv(os.path.join(train_path, "electricity_prices.csv"), columns=electricity_cols, try_parse_dates=True)
df_forecast    = pl.read_csv(os.path.join(train_path, "forecast_weather.csv"), columns=forecast_cols, try_parse_dates=True)
df_historical  = pl.read_csv(os.path.join(train_path, "historical_weather.csv"), columns=historical_cols, try_parse_dates=True)
df_location    = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
#df_location    = pl.read_csv('/kaggle/input/locations/county_lon_lats.csv', columns=location_cols, try_parse_dates=True)
df_target      = df_data.select(target_cols)

schema_data        = df_data.schema
schema_client      = df_client.schema
schema_gas         = df_gas.schema
schema_electricity = df_electricity.schema
schema_forecast    = df_forecast.schema
schema_historical  = df_historical.schema
schema_target      = df_target.schema

### Feature Engineering

In [ ]:
X, y = df_data.drop("target"), df_data.select("target")

X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target)

df_train = to_pandas(X, y)

In [ ]:
#print(df_train.columns.to_list())
#pd.set_option('display.max_rows', None)
#df_train.loc[174]

In [ ]:
df_train = df_train[df_train["target"].notnull() & df_train["year"].gt(2021)]

### HyperParam Optimization

In [ ]:
# study = optuna.create_study(direction='minimize', study_name='Regressor')
# study.optimize(lgb_objective, n_trials=100, show_progress_bar=True)

### Validation

In [ ]:
'''result = cross_validate(
    estimator=lgb.LGBMRegressor(**best_params, random_state=42),
    X=df_train.drop(columns=["target"]), 
    y=df_train["target"],
    scoring="neg_mean_absolute_error",
    cv=MonthlyKFold(1),
)

print(f"Fit Time(s): {result['fit_time'].mean():.3f}")
print(f"Score Time(s): {result['score_time'].mean():.3f}")
print(f"Error(MAE): {-result['test_score'].mean():.3f}")'''
pass

### Training

#### Параметры для LGB

In [ ]:
# #LGB
if is_gpu:
    # max_bin пришлось уменьшить для GPU ниже 250. Вообщедля GPU рекомендуют 63.
    # Не факт что этот новый max_bin хорошо сочетается с другими параметрами
    p1={'device_type': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.06258413085998576, 'colsample_bytree': 0.6527661140701613, 'colsample_bynode': 0.8106858631408332, 'lambda_l1': 5.065645378814257, 'lambda_l2': 9.81159370218779, 'min_data_in_leaf': 192, 'max_depth': 10, 'max_bin': 250}
    p2={'device_type': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.0632167263149817, 'colsample_bytree': 0.6958033941948067, 'colsample_bynode': 0.6030801666196094, 'lambda_l1': 7.137580620471935, 'lambda_l2': 9.348169401713742, 'min_data_in_leaf': 74, 'max_depth': 11, 'max_bin': 220}
    p3={'device_type': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.061236402165228264, 'colsample_bytree': 0.81427095118471, 'colsample_bynode': 0.6097376843527067, 'lambda_l1': 6.360490880385201, 'lambda_l2': 9.954136008333839, 'min_data_in_leaf': 238, 'max_depth': 13, 'max_bin': 180}
    p4={'device_type': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.06753282378023663, 'colsample_bytree': 0.7508715107428325, 'colsample_bynode': 0.6831819500325418, 'lambda_l1': 8.679353563755722, 'lambda_l2': 6.105008696961338, 'min_data_in_leaf': 198, 'max_depth': 15, 'max_bin': 190}
    p5={'device_type': 'gpu', 'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.05129380742257108, 'colsample_bytree': 0.5101576947777211, 'colsample_bynode': 0.8052639518604396, 'lambda_l1': 8.087311995794915, 'lambda_l2': 5.067361158677095, 'min_data_in_leaf': 222, 'max_depth': 8, 'max_bin': 45}
    p6={'device_type': 'gpu', 'n_estimators': 900,'verbose': -1,'objective': 'l2','learning_rate': 0.05689066836106983,'colsample_bytree': 0.8915976762048253,'colsample_bynode': 0.5942203285139224,'lambda_l1': 3.6277555139102864,'lambda_l2': 1.6591278779517808,'min_data_in_leaf' : 186,'max_depth': 9,'max_bin': 100,}
    p_day={'device_type': 'gpu', 'learning_rate': 0.050239193018201116, 'colsample_bytree': 0.7523230869476827, 'colsample_bynode': 0.8016401710184272, 'lambda_l1': 0.804941519994492, 'lambda_l2': 5.420391522845777, 'min_data_in_leaf': 53, 'max_depth': 15, 'max_bin': 250, 'n_estimators': 1367, 'num_leaves': 75, 'feature_fraction': 0.7660656830160648, 'bagging_fraction': 0.8829219702163389, 'bagging_freq': 1}
else:
    p1={'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.06258413085998576, 'colsample_bytree': 0.6527661140701613, 'colsample_bynode': 0.8106858631408332, 'lambda_l1': 5.065645378814257, 'lambda_l2': 9.81159370218779, 'min_data_in_leaf': 192, 'max_depth': 10, 'max_bin': 1800}
    p2={'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.0632167263149817, 'colsample_bytree': 0.6958033941948067, 'colsample_bynode': 0.6030801666196094, 'lambda_l1': 7.137580620471935, 'lambda_l2': 9.348169401713742, 'min_data_in_leaf': 74, 'max_depth': 11, 'max_bin': 530}
    p3={'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.061236402165228264, 'colsample_bytree': 0.81427095118471, 'colsample_bynode': 0.6097376843527067, 'lambda_l1': 6.360490880385201, 'lambda_l2': 9.954136008333839, 'min_data_in_leaf': 238, 'max_depth': 13, 'max_bin': 649}
    p4={'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.06753282378023663, 'colsample_bytree': 0.7508715107428325, 'colsample_bynode': 0.6831819500325418, 'lambda_l1': 8.679353563755722, 'lambda_l2': 6.105008696961338, 'min_data_in_leaf': 198, 'max_depth': 15, 'max_bin': 835}
    p5={'n_estimators': 1000,'verbose': -1,'objective': 'l2','learning_rate': 0.05129380742257108, 'colsample_bytree': 0.5101576947777211, 'colsample_bynode': 0.8052639518604396, 'lambda_l1': 8.087311995794915, 'lambda_l2': 5.067361158677095, 'min_data_in_leaf': 222, 'max_depth': 8, 'max_bin': 97}
    p6={'n_estimators': 900,'verbose': -1,'objective': 'l2','learning_rate': 0.05689066836106983,'colsample_bytree': 0.8915976762048253,'colsample_bynode': 0.5942203285139224,'lambda_l1': 3.6277555139102864,'lambda_l2': 1.6591278779517808,'min_data_in_leaf' : 186,'max_depth': 9,'max_bin': 813,}

#### Параметры для catboost

In [ ]:
if is_gpu:
    # Параметры для catboost c GPU
    c1 = {
        'learning_rate': 0.06258413085998576,
        'depth': 10,
        'l2_leaf_reg': 8,
        'border_count': 211,
        'random_strength': 6,
        'bagging_temperature': 0.13029094645654574,
        'iterations': 1500,
        'eval_metric': 'RMSE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1'],
        'task_type': 'GPU'
    }
    
    c2 = {
        'learning_rate': 0.08766355644863072,
        'depth': 10,
        'l2_leaf_reg': 8,
        'border_count': 231,
        'random_strength': 5,
        'bagging_temperature': 0.20294125980762928,
        'iterations': 1500,
        'eval_metric': 'RMSE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1'],
        'task_type': 'GPU'
    }
    
    c3 = {
        'learning_rate': 0.14331003896351277,
        'depth': 7,
        'l2_leaf_reg': 10,
        'border_count': 162,
        'random_strength': 2,
        'bagging_temperature': 0.3016878017499466,
        'iterations': 1500,
        'eval_metric': 'RMSE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1'],
        'task_type': 'GPU'
    }
    
    c4 = {
        'learning_rate': 0.13382144579754543,
        'depth': 7,
        'l2_leaf_reg': 10,
        'border_count': 179,
        'random_strength': 5,
        'bagging_temperature': 0.30199258643156335,
        'iterations': 1500,
        'eval_metric': 'RMSE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1'],
        'task_type': 'GPU'
    }

    c5 = {
        'learning_rate': 0.12358952478027072,
        'depth': 11,
        'l2_leaf_reg': 8,
        'border_count': 191,
        'random_strength': 3,
        'bagging_temperature': 0.41774414265586035,
        'iterations': 1500,
        'eval_metric': 'RMSE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1'],
        'task_type': 'GPU'
    }
    
    c6 = {
        'learning_rate': 0.13767418996955944,
        'depth': 7,
        'l2_leaf_reg': 10,
        'border_count': 182,
        'random_strength': 3,
        'bagging_temperature': 0.42796507669281386,
        'iterations': 500,
        'eval_metric': 'RMSE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1'],
        'task_type': 'GPU'
    }
else:
    # Параметры для catboost без GPU
    c1 = {
        'learning_rate': 0.06258413085998576,
        'depth': 10,
        'l2_leaf_reg': 8,
        'border_count': 211,
        'random_strength': 6,
        'bagging_temperature': 0.13029094645654574,
        'iterations': 1500,
        'eval_metric': 'MAE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1']
    }
    
    c2 = {
        'learning_rate': 0.08766355644863072,
        'depth': 10,
        'l2_leaf_reg': 8,
        'border_count': 231,
        'random_strength': 5,
        'bagging_temperature': 0.20294125980762928,
        'iterations': 1500,
        'eval_metric': 'MAE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1']
    }
    
    c3 = {
        'learning_rate': 0.14331003896351277,
        'depth': 7,
        'l2_leaf_reg': 10,
        'border_count': 162,
        'random_strength': 2,
        'bagging_temperature': 0.3016878017499466,
        'iterations': 1500,
        'eval_metric': 'MAE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1']
    }
    
    c4 = {
        'learning_rate': 0.13382144579754543,
        'depth': 7,
        'l2_leaf_reg': 10,
        'border_count': 179,
        'random_strength': 5,
        'bagging_temperature': 0.30199258643156335,
        'iterations': 1500,
        'eval_metric': 'MAE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1']
    }
    
    c5 = {
        'learning_rate': 0.12358952478027072,
        'depth': 11,
        'l2_leaf_reg': 8,
        'border_count': 191,
        'random_strength': 3,
        'bagging_temperature': 0.41774414265586035,
        'iterations': 1500,
        'eval_metric': 'MAE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1']
    }
    
    c6 = {
        'learning_rate': 0.13767418996955944,
        'depth': 7,
        'l2_leaf_reg': 10,
        'border_count': 182,
        'random_strength': 3,
        'bagging_temperature': 0.42796507669281386,
        'iterations': 500,
        'eval_metric': 'MAE',
        'cat_features': ['county', 'is_business', 'product_type', 'is_consumption', 'category_1']
    }

#### Класс моделей

In [ ]:
# Класс обертка для моделей. Хранит различные пареметры для обучения моделей.
# Например диапазоны данных на которых учить модель, как часто обучать заново и другие параметры
class Models:
    
    # Инициализирует параметры обучения
    # init_model - готовый объект модели для обучения
    def __init__(self):
        # Инициализируем словари
        # Ключами во всех словарях будет имя модели
        
        # Словарь с моделями
        self.models =  dict()
        
        # Словарь с описанием периодов обучения модели
        # пока это матрица. из двух столбцов и двух строк в каждой строке описание периода
        # первая колонка на сколько data_block_id в конеце обучени отстоит от доступного конца данных
        # вторая колонка сколько data_block_id будет в периоде на котором обучаемся.
        # data_block_id могут быть эквивалентны дням, но могут и отличаться, если данные будут подавать блоками не равными дням
        # либо если будут разрывы в данных. Но они точно будут эквиваленты циклам предсказания
        self.data_block_id_intervals  = dict()
        
        # Если 1, то модель предназначена для предсказания потребления электричества
        # Если 0, то модель предназначена для предсказания производства электричества
        self.is_consumption = dict()
        
        # Раз во сколько иттераций обучат модель
        self.learn_again_period = dict()
        
        # Смещение для начала обучения модели. Добавляется к номеру итерации сабмита.
        # Скажем если смещение 6 номер итерации 1, а обучаемся раз в чем итераций. То обучение будет в первуже итерацию сабмита.
        self.learn_again_offset = dict()
        
        # Время в секундах сколько заняло последнее обучение модели
        self.last_learn_time = dict()
    
    # Добавляет еще одну модель
    # model_name - название модели
    # new_model - объект модели
    def add_model(self, model_name, new_model, is_consumption, data_block_id_intervals
                  ,learn_again_period, learn_again_offset):
        
        self.models[model_name] = new_model
        self.is_consumption[model_name] = is_consumption
        self.data_block_id_intervals[model_name] = data_block_id_intervals
        self.learn_again_period[model_name] = learn_again_period
        self.learn_again_offset[model_name] = learn_again_offset
        self.last_learn_time[model_name] = 0
        
    # Обучает модель
    # model_name - название модели
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    def fit_one_model(self, model_name, df_train):
        print('fit model:', model_name)
        self.models[model_name].fit(
            X=df_train.drop(columns=["target", "data_block_id"]),
            y=df_train["target"]
        )
    
    # Делает предсказания от отдельной модели.
    # model_name - название модели
    # X - признаки на которых нужно сделать предсказание
    def predict_one_model(self, model_name, X):
        print('predict model:', model_name)
        y = (self.models[model_name]
             .predict(X.drop(columns=["data_block_id"])).clip(0)
            )
        return(y)
    
    # Обучает все добавленные модели для которых наступило время их обучения
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    # itter_n - номер иттерации в сабмите.
    # если itter_n равен -1. Значит первоначальное обучение и обучаем всем модели
    def fit(self, df_train, itter_n):
        max_block_id = df_train["data_block_id"].max()
        # Перебираем все модели
        for m_name in self.learn_again_period:
            # Либо сказали все модели учить
            if ((itter_n == -1)
                # Либо учим если номер итераиции в сабмите плюс смещение делится на цело на период обучения
                or (((itter_n + self.learn_again_offset[m_name])
                     % self.learn_again_period[m_name]) == 0)):
                
                print('fit model:', m_name)
                d_i = self.data_block_id_intervals[m_name]
                print(d_i[0][0], d_i[0][1], d_i[1][0], d_i[1][1])
                
                # Выделяем интервалы для обучения1
                # df_train_int = df_train[df_train['is_consumption']==0]
                df_train_int = df_train[
                    # До какого data_block_id учим первый блок
                    (((df_train['data_block_id']<=max_block_id-d_i[0][0])
                     # C какого data_block_id учим первый блок
                     &(df_train['data_block_id']>(max_block_id-d_i[0][0]-d_i[0][1])))
                    # До какого data_block_id учим второй блок
                    |((df_train['data_block_id']<=max_block_id-d_i[1][0])
                      # С какого data_block_id учим второй блок
                      &(df_train['data_block_id']>(max_block_id-d_i[1][0]-d_i[1][1]))))
                &df_train[df_train['is_consumption']==self.is_consumption[model_name]]
                ]
                
                print(df_train_int['data_block_id'].unique())
    
    # Делает предсказание всеми добавленными моделями и сводит их в одно предсказание
    def predict(self, df_train):
        pass

        

In [ ]:
'''
%%time
X, y = df_data.drop("target"), df_data.select("target")
X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target)
df_train = to_pandas(X, y)
df_train = df_train[df_train["target"].notnull() & df_train["year"].gt(2021)]
'''
pass

In [ ]:
# sorted(df_train['data_block_id'].unique())
# df_train['data_block_id'].unique()
df_train['is_consumption'].unique()

In [ ]:
models = Models()

In [ ]:
models.add_model(
    model_name = 'first',
    new_model = lgb.LGBMRegressor(**p5, verbosity=-1, random_state=42),
    is_consumption = 0,
    data_block_id_intervals = [[0,10],[30,20]],
    learn_again_period = 7,
    learn_again_offset = 1
)

In [ ]:
models.fit(df_train=df_train, itter_n=-1)

In [ ]:
%%time
# models.fit_one_model(model_name='first', df_train=df_train)

#### Обучение моделей

In [ ]:
%%time

if only_one_model:
    # Если только одна модель
    model = lgb.LGBMRegressor(**p5, verbosity=-1, random_state=42)
    model.fit(
        X=df_train[df_train['is_consumption']==1].drop(columns=["target", "data_block_id"]),
        y=df_train[df_train['is_consumption']==1]["target"]
    )
else:
    model = VotingRegressor([
        ('lgb_1', lgb.LGBMRegressor(**p1, random_state=42)), 
        ('lgb_2', lgb.LGBMRegressor(**p2, random_state=42)), 
        ('lgb_3', lgb.LGBMRegressor(**p3, random_state=42)), 
        ('lgb_4', lgb.LGBMRegressor(**p4, random_state=42)), 
        ('lgb_5', lgb.LGBMRegressor(**p5, random_state=42)), 
    ])
    
    model.fit(
        X=df_train[df_train['is_consumption']==1].drop(columns=["target", "data_block_id"]),
        y=df_train[df_train['is_consumption']==1]["target"]
    )

In [ ]:
%%time

if not(only_one_model):
    # Если только одна модель
    model_solar = VotingRegressor([
        ('catboost_1', cb.CatBoostRegressor(**c1, verbose=False, random_state=42)),
        ('catboost_2', cb.CatBoostRegressor(**c2, verbose=False, random_state=42)),
        ('catboost_3', cb.CatBoostRegressor(**c3, verbose=False, random_state=42)),
        ('catboost_4', cb.CatBoostRegressor(**c4, verbose=False, random_state=42)),
        ('catboost_5', cb.CatBoostRegressor(**c5, verbose=False, random_state=42))
    ])
    model_solar.fit(
        X=df_train[df_train['is_consumption']==0].drop(columns=["target", "data_block_id"]),
        y=df_train[df_train['is_consumption']==0]["target"]
    )
pass

In [ ]:
#if is_local:
#    dump(model_solar, 'model_solar.joblib')
#    dump(model, 'model_lgbm.joblib')

Расскоментировать при необходимости загрузку ранее сохраненных моделей

In [ ]:
# Для загрузки локально
#model_solar = load('model_solar.joblib')
#model = load('model_lgbm.joblib')

# Для загрузки на kaggle
#model_solar = load('/kaggle/input/enefit/model_solar.joblib')
#model = load('/kaggle/input/enefit/model_lgbm.joblib')

## Prediction

In [ ]:
if is_local:
    # Если выполняем локально, а не сабмитим на кагл,
    # то выбираем другое имя для файла submission.csv.
    # Потому что в submission.csv записать прав нет и вылетает по ошибке
    submission_name = 'submission_loc.csv'
else:
    submission_name = 'submission.csv'

### Содержимое public_timeseries_testing_util.py

С необходимыми праками. Решил не импортировать его. а прямо тут. Так удобнее переносить на kaggle

In [ ]:
'''
An unlocked version of the timeseries API intended for testing alternate inputs.
Mirrors the production timeseries API in the crucial respects, but won't be as fast.

ONLY works afer the first three variables in MockAPI.__init__ are populated.
'''

from typing import Sequence, Tuple


class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = ['example_test_files/test.csv',
                                   'example_test_files/revealed_targets.csv', 
                                   'example_test_files/client.csv',
                                   'example_test_files/historical_weather.csv',
                                   'example_test_files/forecast_weather.csv',
                                   'example_test_files/electricity_prices.csv',
                                   'example_test_files/gas_prices.csv',
                                   'example_test_files/sample_submission.csv']
        self.group_id_column: str = 'data_block_id'
        self.export_group_id_column: bool = False
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open(submission_name, 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()


### Инициализация иттераций сабмита

In [ ]:
if is_local:
    # После этого можно имитировать локально загрузку при собмите на большом числе итераций
    # А не только четыре иттерации на 4 дня как в стандартной имитайии на кагле
    env = make_env()
else:
    # загружаем оригинальную библиотеку для сабмита
    import enefit
    env = enefit.make_env()

iter_test = env.iter_test()

### Цикл сабмита

In [ ]:
# Тренирует модели заново
def train_models():
    # Подготавливаем данные
    global  X, y, df_client, df_gas, df_electricity, df_forecast,\
            df_historical, df_location, df_target, df_train, model, model_solar

    print('Начали обучение моделей')
    X, y = df_data.drop("target"), df_data.select("target")
    X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target)
    df_train = to_pandas(X, y)
    df_train = df_train[df_train["target"].notnull() & df_train["year"].gt(2021)]

    # Обучаем модели
    print('LGB')
    model.fit(
        X=df_train[df_train['is_consumption']==1].drop(columns=["target", "data_block_id"]),
        y=df_train[df_train['is_consumption']==1]["target"]
    )
    if not(only_one_model):
        print('Catboost')
        model_solar.fit(
            X=df_train[df_train['is_consumption']==0].drop(columns=["target", "data_block_id"]),
            y=df_train[df_train['is_consumption']==0]["target"]
        )
    print('Завершили обучение моделей')

In [ ]:
# Находим последний data_block_id в обучающих данных
max_train_data_block_id = df_data["data_block_id"].max()
# Устанавливаем первый data_block_id для теста следущим за тренировочным
cur_test_data_block_id = max_train_data_block_id + 1
cur_test_data_block_id

In [ ]:
%%time
count = 0

# Основной цикл для обработки данных тестового набора
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    # Переименование столбца для удобства
    test = test.rename(columns={"prediction_datetime": "datetime"})
    if is_local:
        # Если выполняем локально, то преобразуем некоторые типы данных
        # На кагле (а может и в линуксе) они и так преобразуются, но на виновс локально
        # не преобразуются и выдетают по ощибке
        test['datetime'] = pd.to_datetime(test['datetime'])
        client['date'] = pd.to_datetime(client['date'])
        gas_prices['origin_date'] = pd.to_datetime(gas_prices['origin_date'])
        gas_prices['forecast_date'] = pd.to_datetime(gas_prices['forecast_date'])
        electricity_prices['origin_date'] = pd.to_datetime(electricity_prices['origin_date'])
        electricity_prices['forecast_date'] = pd.to_datetime(electricity_prices['forecast_date'])
        forecast_weather['origin_datetime'] = pd.to_datetime(forecast_weather['origin_datetime'])
        forecast_weather['forecast_datetime'] = pd.to_datetime(forecast_weather['forecast_datetime'])
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        revealed_targets['datetime'] = pd.to_datetime(revealed_targets['datetime'])
        
    # Добавляем колонку заполненную следующим data_block_id
    test["data_block_id"] = cur_test_data_block_id
    revealed_targets["data_block_id"] = cur_test_data_block_id
    
    df_test            = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_new_client      = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_new_gas         = pl.from_pandas(gas_prices[gas_cols], schema_overrides=schema_gas)
    df_new_electricity = pl.from_pandas(electricity_prices[electricity_cols], schema_overrides=schema_electricity)
    df_new_forecast    = pl.from_pandas(forecast_weather[forecast_cols], schema_overrides=schema_forecast)
    df_new_historical  = pl.from_pandas(historical_weather[historical_cols], schema_overrides=schema_historical)
    df_new_target      = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    df_new_data        = pl.from_pandas(revealed_targets[df_data_cols], schema_overrides=schema_data)
    # Объединение новых данных с существующими и удаление дубликатов
    df_client          = pl.concat([df_client, df_new_client]).unique(subset=["county", "is_business", "product_type", "date"], maintain_order=True)
    df_gas             = pl.concat([df_gas, df_new_gas]).unique(subset=["forecast_date"], maintain_order=True)
    df_electricity     = pl.concat([df_electricity, df_new_electricity]).unique(subset=["forecast_date"], maintain_order=True)
    df_forecast        = pl.concat([df_forecast, df_new_forecast]).unique()
    df_historical      = pl.concat([df_historical, df_new_historical]).unique()
    df_target          = pl.concat([df_target, df_new_target]).unique()
    df_data            = pl.concat([df_data, df_new_data]).unique()
    
    # Применение функции инженерии признаков и преобразование данных обратно в pandas
    X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target)
    X_test = to_pandas(X_test)
    
    # Прогнозирование с использованием модели и ограничение предсказаний нулем
    test['target'] = model.predict(X_test.drop(columns=["data_block_id"])).clip(0)
    
    if not(only_one_model):
        test['target_solar'] = model_solar.predict(X_test.drop(columns=["data_block_id"])).clip(0)
        
        # Замена прогнозов для непотребляющих клиентов на прогнозы солнечной энергии
        test.loc[test['is_consumption']==0, "target"] = test.loc[test['is_consumption']==0, "target_solar"]    
    
    # Обновление целевых значений в примере предсказания
    sample_prediction["target"] = test['target']
    
    # Отправка предсказаний в среду выполнения
    env.predict(sample_prediction)

    count += 1
    print(count)

    if is_local:
        if (((count % learn_again_period) == 0)
            and (count>=100)):
            # Учим модели заново, с учетом новых данных но только после
            # итерации 100. Потому что интересует как ведет себя моделья через два месяца
            # после обучения
            train_models()
    else:
        # На реальном cабмите начинаем тренировать модели сразу.
        # Кроме того, если работаем больше 8 часов прекращаем тренировать модели на реальном сабмите
        # Потому что после 9 часов сабмит вылетит по таймауту
        cur_time = time.time()
        if ((cur_time - notebook_starttime) < (8*60*60)):
            if ((count % learn_again_period) == 0):
                train_models()
        else:
            print('Не тренеруем модель, превышено время выполнения ноутбука:', (cur_time - notebook_starttime))
    
    # Переходим к следующему data_block_id на итерации тестов
    cur_test_data_block_id += 1

## Анализ предсказания

### Подсчет скора

In [ ]:
if is_local:
    # Загружаем предсказания
    submission = pd.read_csv(submission_name)
    # Загружаем истинные значения
    revealed_targets = pd.read_csv(os.path.join(test_path, "revealed_targets.csv"))

    mae = mean_absolute_error(revealed_targets['target'] , submission['target'])
    print(f'MAE: {mae}')
    
    # Подготовим данные для анализа изменения ошибки предсказания по мере удаления от времени завершения обучения
    compare = pd.DataFrame(revealed_targets['data_block_id'])
    compare['target'] = revealed_targets['target']
    compare['predict'] = submission['target']
    compare['abs_err'] = abs(compare['predict'] - compare['target'])
    compare['err'] = compare['predict'] - compare['target']
    
    compare_600 = compare[compare['data_block_id'] > 600]
    # Выводим MAE для data_block_id > 600
    mae_600 = mean_absolute_error(compare_600['target'], compare_600['predict'])
    
    print(f'MAE после 600: {mae_600}')

### График MAE по дням предсказания

In [ ]:
# выводит график средних ошибок сгруппированных по дням (точнее для блоков данных для предсказаний которые в целом эквиваленты дням)
def print_err(err_name, err_lable, err_title):
    # Группируем по data_block_id, то есть по дням и считаем отдельно для каждого дня предсказания MAE
    grouped_compare = compare.groupby('data_block_id').mean().reset_index()
    # Делаем скользящую среднюю
    grouped_compare['rolling_mean'] = grouped_compare[err_name].rolling(window=30, min_periods=1).mean()
    
    
    # Plotting the mean absolute errors
    plt.figure(figsize=(10, 8))
    #plt.bar(grouped_compare['data_block_id'], grouped_compare['abs_err'])
    plt.bar(grouped_compare['data_block_id'], grouped_compare[err_name], label=err_lable)
    plt.plot(grouped_compare['data_block_id'],
             grouped_compare['rolling_mean'],
             label='Rolling Mean (window=30)',
             color='orange',
             linestyle='-', linewidth=2)
    plt.xlabel('data_block_id')
    plt.ylabel(err_lable)
    plt.title(err_title)
    plt.legend()
    
    # Set ticks every 10 data_block_id
    tick_positions_y = np.arange(-40, max(grouped_compare[err_name]) + 1, 10)
    plt.yticks(tick_positions_y)
    plt.grid(True)
    plt.show()

In [ ]:
if is_local:
    print_err(err_name='abs_err',
              err_lable='Mean Absolute Error',
              err_title='Mean Absolute Error by data_block_id')

In [ ]:
if is_local:
    compare = compare[compare["data_block_id"] > 600]
    print_err(err_name='err',
              err_lable='Mean Error (predict - target)',
              err_title='Mean Error by data_block_id')